<a href="https://colab.research.google.com/github/Toan02Ky-UIT/ComputerVisionProject/blob/main/Transfer_Agument_s%E1%BB%91_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

path = kagglehub.dataset_download("mohamedmaher5/vehicle-classification")
path

100%|██████████| 827M/827M [00:08<00:00, 98.8MB/s]

Extracting files...


'/root/.cache/kagglehub/datasets/mohamedmaher5/vehicle-classification/versions/1'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!mkdir -p "/content/drive/MyDrive/vehicle"

In [ ]:
!cp -r "{path}" "/content/drive/MyDrive/vehicle"

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import copy
import time
import os


In [ ]:
data_dir = "/content/drive/MyDrive/vehicle/1/Vehicles"
batch_size = 32
img_size = 224

train_tf = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(0.2, 0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])

test_tf = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])

full_dataset = datasets.ImageFolder(data_dir)

n = len(full_dataset)
train_len = int(n * 0.8)
val_len = int(n * 0.1)
test_len = n - train_len - val_len

train_subset, val_subset, test_subset = random_split(full_dataset, [train_len, val_len, test_len])

train_subset.dataset = datasets.ImageFolder(data_dir, transform=train_tf)
val_subset.dataset  = datasets.ImageFolder(data_dir, transform=test_tf)
test_subset.dataset = datasets.ImageFolder(data_dir, transform=test_tf)

train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

num_classes = len(full_dataset.classes)

In [ ]:
def get_model(name, num_classes):
    if name == "resnet50":
        model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        in_f = model.fc.in_features
        model.fc = nn.Linear(in_f, num_classes)

    elif name.startswith("effnet"):
        eff = {
            "effnet_b0": models.efficientnet_b0,
            "effnet_b1": models.efficientnet_b1,
            "effnet_b2": models.efficientnet_b2,
            "effnet_b3": models.efficientnet_b3,
        }
        model = eff[name](weights="IMAGENET1K_V1")
        in_f = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_f, num_classes)

    for param in model.parameters():
        param.requires_grad = False
    for param in model.fc.parameters() if name == "resnet50" else model.classifier.parameters():
        param.requires_grad = True

    return model

In [ ]:
def train_model(model, name):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=3)
    criterion = nn.CrossEntropyLoss()

    best_weights = copy.deepcopy(model.state_dict())
    best_acc = 0
    best_loss = float("inf")
    es_loss = float("inf")
    patience, stop_counter = 7, 0

    for epoch in range(200):
        model.train()
        train_loss, correct, total = 0, 0, 0

        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            out = model(imgs)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * imgs.size(0)
            _, preds = torch.max(out, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        train_loss /= total

        model.eval()
        correct, total, val_loss = 0, 0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                out = model(imgs)
                val_loss += criterion(out, labels).item() * imgs.size(0)
                _, preds = torch.max(out, 1)
                total += labels.size(0)
                correct += (preds == labels).sum().item()

        val_acc = correct / total
        val_loss /= total
        scheduler.step(val_loss)

        print(f"{name} → Epoch {epoch+1} | train acc={train_acc:.4f} | train loss={train_loss:.4f} | "
          f"val acc={val_acc:.4f} | val loss={val_loss:.4f} | lr={optimizer.param_groups[0]['lr']:.6f}")


        if (val_acc > best_acc) or (val_acc == best_acc and val_loss < best_loss):
            best_acc = val_acc
            best_loss = val_loss
            best_weights = copy.deepcopy(model.state_dict())
            torch.save(best_weights, f"/content/drive/MyDrive/vehicle/{name}_best.pt")
            print("  ✓ Saved best checkpoint")

        if val_loss < es_loss:
            es_loss = val_loss
            stop_counter = 0
        else:
            stop_counter += 1
            if stop_counter >= patience:
                print("  ⛔ Early stopping")
                break

    model.load_state_dict(best_weights)
    return model


In [ ]:
def evaluate(model, name):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            out = model(imgs)
            _, preds = torch.max(out, 1)
            y_true.extend(labels.numpy())
            y_pred.extend(preds.cpu().numpy())

    print(f"\n=== {name} — Metrics ===")
    print(classification_report(y_true, y_pred, target_names=full_dataset.classes, digits=4))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

In [ ]:
model_list = ["resnet50", "effnet_b3"]

for m in model_list:
    print(f"\n================ RUN MODEL: {m} ================")
    model = get_model(m, num_classes)
    trained = train_model(model, m)
    evaluate(trained, m)



================ RUN MODEL: resnet50 ================
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 131MB/s]
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


resnet50 → Epoch 1 | train acc=0.9087 | train loss=0.5837 | val acc=0.9749 | val loss=0.1707 | lr=0.001000
  ✓ Saved best checkpoint
resnet50 → Epoch 2 | train acc=0.9700 | train loss=0.1732 | val acc=0.9875 | val loss=0.0996 | lr=0.001000
  ✓ Saved best checkpoint
resnet50 → Epoch 3 | train acc=0.9736 | train loss=0.1221 | val acc=0.9928 | val loss=0.0631 | lr=0.001000
  ✓ Saved best checkpoint
resnet50 → Epoch 4 | train acc=0.9783 | train loss=0.0943 | val acc=0.9875 | val loss=0.0553 | lr=0.001000
resnet50 → Epoch 5 | train acc=0.9796 | train loss=0.0826 | val acc=0.9928 | val loss=0.0433 | lr=0.001000
  ✓ Saved best checkpoint
resnet50 → Epoch 6 | train acc=0.9850 | train loss=0.0710 | val acc=0.9946 | val loss=0.0413 | lr=0.001000
  ✓ Saved best checkpoint
resnet50 → Epoch 7 | train acc=0.9886 | train loss=0.0600 | val acc=0.9946 | val loss=0.0382 | lr=0.001000
  ✓ Saved best checkpoint
resnet50 → Epoch 8 | train acc=0.9846 | train loss=0.0614 | val acc=0.9964 | val loss=0.0311 | 

100%|██████████| 47.2M/47.2M [00:00<00:00, 179MB/s]
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


effnet_b3 → Epoch 1 | train acc=0.8687 | train loss=0.7197 | val acc=0.9677 | val loss=0.2707 | lr=0.001000
  ✓ Saved best checkpoint
effnet_b3 → Epoch 2 | train acc=0.9401 | train loss=0.2726 | val acc=0.9767 | val loss=0.1578 | lr=0.001000
  ✓ Saved best checkpoint
effnet_b3 → Epoch 3 | train acc=0.9501 | train loss=0.2095 | val acc=0.9713 | val loss=0.1356 | lr=0.001000
effnet_b3 → Epoch 4 | train acc=0.9528 | train loss=0.1809 | val acc=0.9749 | val loss=0.1137 | lr=0.001000
effnet_b3 → Epoch 5 | train acc=0.9546 | train loss=0.1661 | val acc=0.9731 | val loss=0.1019 | lr=0.001000
effnet_b3 → Epoch 6 | train acc=0.9555 | train loss=0.1523 | val acc=0.9749 | val loss=0.0935 | lr=0.001000
effnet_b3 → Epoch 7 | train acc=0.9604 | train loss=0.1378 | val acc=0.9749 | val loss=0.0884 | lr=0.001000
effnet_b3 → Epoch 8 | train acc=0.9631 | train loss=0.1317 | val acc=0.9749 | val loss=0.0900 | lr=0.001000
effnet_b3 → Epoch 9 | train acc=0.9609 | train loss=0.1256 | val acc=0.9803 | val lo